# Gallery Example: M/H₂/1 Queue (Hyperexponential Service)

This example demonstrates an M/H₂/1 queueing system:
- **Arrivals**: Poisson process (Exponential inter-arrival times)
- **Service**: Hyperexponential service times (more variable than exponential)
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

The Hyperexponential service distribution has higher variance than exponential, representing inconsistent service times (SCV=4).

In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mhyp1():    """Create M/H₂/1 queueing model"""    model = Network('M/H/1')        # Block 1: nodes    source = Source(model, 'mySource')    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)    sink = Sink(model, 'mySink')        # Block 2: classes    oclass = OpenClass(model, 'myClass')    # Exponential arrivals with rate λ=1    source.set_arrival(oclass, Exp(1))    # Hyperexponential service with mean=0.5 and SCV=4    queue.set_service(oclass, HyperExp.fit_mean_and_scv(0.5, 4))        # Block 3: topology    P = model.init_routing_matrix()    P.add_route(oclass, source, queue, 1.0)    P.add_route(oclass, queue, sink, 1.0)    model.link(P)        return model# Create the modelmodel = gallery_mhyp1()

## Theoretical Analysis for M/H₂/1

For M/H₂/1 with:
- **Arrival rate**: λ = 1 (Exponential)
- **Service time**: Hyperexponential with mean=0.5, SCV=4
- **Utilization**: ρ = λ × E[S] = 1 × 0.5 = 0.5

The Hyperexponential service characteristics:
- **Mean**: 0.5
- **Squared Coefficient of Variation**: C²ₛ = 4
- **Variance**: 4 × (0.5)² = 1.0 (much higher than exponential variance = 0.25)

Higher service variability leads to worse performance than M/M/1.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)

In [ ]:
# Compare service time variability effects
print("\n=== Service Variability Comparison ===")

def create_service_model(dist_type, scv=None, order=None):
    """Create model with different service distributions"""
    model_svc = Network(f'M/{dist_type}/1')
    source = Source(model_svc, 'Source')
    queue = Queue(model_svc, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model_svc, 'Sink')
    
    oclass = OpenClass(model_svc, 'Class')
    source.set_arrival(oclass, Exp(1))
    
    if dist_type == 'D':
        # Deterministic service
        queue.set_service(oclass, Det(0.5))
    elif dist_type == 'E':
        # Erlang service
        queue.set_service(oclass, Erlang.fit_mean_and_order(0.5, order or 2))
    elif dist_type == 'M':
        # Exponential service
        queue.set_service(oclass, Exp(2))
    elif dist_type == 'H':
        # Hyperexponential service
        queue.set_service(oclass, HyperExp.fit_mean_and_scv(0.5, scv or 4))
    
    P = model_svc.init_routing_matrix()
    P.add_route(oclass, source, queue, 1.0)
    P.add_route(oclass, queue, sink, 1.0)
    model_svc.link(P)
    
    return model_svc

# Compare different service distributions
distributions = [
    ('D', 0.0, None, 'Deterministic'),
    ('E', 0.5, 2, 'Erlang-2'),
    ('M', 1.0, None, 'Exponential'),
    ('H', 4.0, None, 'Hyperexp')
]

print("Service Type    | C²ₛ | Response Time | Queue Length")
print("-" * 50)

for dist_type, cv_sq, order, name in distributions:
    try:
        if dist_type == 'D':
            model_dist = create_service_model(dist_type)
        elif dist_type == 'E':
            model_dist = create_service_model(dist_type, order=order)
        elif dist_type == 'H':
            model_dist = create_service_model(dist_type, scv=cv_sq)
        else:
            model_dist = create_service_model(dist_type)
        
        solver = MVA(model_dist)
        avg_table = solver.avg_table()
        
        resp_time = float(avg_table.iloc[1, 2])
        queue_length = float(avg_table.iloc[1, 3])
        
        print(f"{name:12s}    | {cv_sq:.1f} |     {resp_time:.4f}    |    {queue_length:.4f}")
    except Exception as e:
        print(f"{name:12s}    | {cv_sq:.1f} | Error: {str(e)[:20]}...")

print("\nObservation: Performance degrades as service variability (C²ₛ) increases.")

In [ ]:
# Pollaczek-Khinchine formula validation
print("\n=== Pollaczek-Khinchine Formula Validation ===")

# For M/G/1, the mean response time is given by:
# W = E[S] + (λ × E[S²]) / (2 × (1 - ρ))
# where E[S²] = Var[S] + (E[S])²

lambda_rate = 1.0
mean_service = 0.5
rho = lambda_rate * mean_service
scv = 4.0
var_service = scv * (mean_service ** 2)
second_moment = var_service + (mean_service ** 2)

# Theoretical calculation
w_theory = mean_service + (lambda_rate * second_moment) / (2 * (1 - rho))
l_theory = lambda_rate * w_theory

# Simulation results
solver = MVA(model)
avg_table = solver.avg_table()
w_simulation = float(avg_table.iloc[1, 2])
l_simulation = float(avg_table.iloc[1, 3])

print(f"M/H₂/1 with λ=1, E[S]=0.5, C²ₛ=4:")
print(f"")
print(f"Response Time:")
print(f"  Theory (P-K):     {w_theory:.4f}")
print(f"  Simulation:       {w_simulation:.4f}")
print(f"  Relative Error:   {abs(w_theory - w_simulation) / w_theory * 100:.2f}%")
print(f"")
print(f"Queue Length:")
print(f"  Theory (Little):  {l_theory:.4f}")
print(f"  Simulation:       {l_simulation:.4f}")
print(f"  Relative Error:   {abs(l_theory - l_simulation) / l_theory * 100:.2f}%")

print(f"\nPollaczek-Khinchine components:")
print(f"  E[S] = {mean_service}")
print(f"  Var[S] = {var_service}")
print(f"  E[S²] = {second_moment}")
print(f"  ρ = {rho}")
print(f"  Waiting time = {w_theory - mean_service:.4f}")
print(f"  Service time = {mean_service}")